### Code Hist.

 - CODE : KIER Data_전처리 (결측 제거)  
 - DESC  
   &ensp; : int_Domain 코드의 변경에 따라,   
   &emsp; 보간된 KIER INST Data를 기반으로 ACCU Data를 재구성  
 - DATE  
   &ensp; 2024-08-05 Created  

## 01. Code

### 01-01. Init

#### 01-01-01. Init_Module Import

#### Module import

In [65]:
#region Import_Basic Module
## Basic
import os, sys, warnings
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.path.dirname(os.path.abspath('./__file__'))
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('./__file__'))))
warnings.filterwarnings('ignore')

import numpy as np, pandas as pd
from pandas import DataFrame, Series
pd.options.display.float_format = '{:.10f}'.format

import math, random

## Datetime
import time, datetime as dt
from datetime import datetime, date, timedelta

## glob
import glob, requests, json
from glob import glob

## 시각화
import matplotlib.pyplot as plt, seaborn as sns
# %matplotlib inline
plt.rcParams['figure.figsize'] = [10, 8]

from scipy import stats

## Split, 정규화
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# K-Means 알고리즘
from sklearn.cluster import KMeans, MiniBatchKMeans

# Clustering 알고리즘의 성능 평가 측도
from sklearn import metrics
from sklearn.metrics import homogeneity_score, completeness_score, v_measure_score, adjusted_rand_score, silhouette_score, rand_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.metrics.cluster import contingency_matrix

## For Web
import urllib
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

import tqdm
from tqdm.notebook import tqdm
#endregion Import_Basic Module

In [66]:
## Import_DL
str_tar = "tf"
## For Torch
if str_tar == "torch":
    import torch, torch.nn as nn
    from torch.nn.utils import weight_norm
    print("Torch Imported")
## For TF
elif str_tar == "tf":
    import tensorflow as tf, tensorflow_addons as tfa
    from keras.callbacks import EarlyStopping, ModelCheckpoint
    from keras.models import Sequential, load_model
    from keras_flops import get_flops
    print("Tensorflow Imported")
else:
    print("Error : Cannot be used except for Keywords")
    print(" : torch / tf")

Tensorflow Imported


In [67]:
## Import_Local
from Src_Dev_Common import Data_Datetime as com_date, Data_Preprocessing as com_Prep, KMA_Weather as com_KMA, KECO_AirKor as com_KECO, KASI_Holiday as com_Holi, KIER_Usage_M02 as com_KIER_M02

### 01-01-02. Config (Directory, Params)

In [68]:
## Init_config
SEED = 42

np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)
os.environ["PYTHONHASHSEED"], os.environ['TF_DETERMINISTIC_OPS'] = str(SEED), "1"

In [69]:
## Define Todate str
str_now_ymd = pd.datetime.now().date()
str_now_y, str_now_m, str_now_d = pd.datetime.now().year, pd.datetime.now().month, pd.datetime.now().day
str_now_hr, str_now_min = pd.datetime.now().hour, pd.datetime.now().minute

print(pd.datetime.now())
print(str(str_now_y) + " / " + str(str_now_m)  + " / " + str(str_now_d))
print(str(str_now_hr) + " : " + str(str_now_min))

2024-08-06 21:46:35.553125
2024 / 8 / 6
21 : 46


In [70]:
## Dict_Domain
int_domain = 5

## Domain, ACCU/INST Column
str_domain, str_col_accu, str_col_inst = com_KIER_M02.create_domain_str(int_domain)
## Directory Root
str_dirData, str_dir_raw, str_dir_cleansed, str_dirName_bld, str_dirName_h = com_KIER_M02.create_dir_str(str_domain)

## File
str_fileRaw, str_fileRaw_hList = str('KIER_RAW_' + str_domain + '_H_ID_Adopted.csv'), str('KIER_hList_Common.csv')

print(str(os.listdir(str_dirData)) + "\n")
print(os.listdir(str_dirName_h))

5 : GAS
['.ipynb_checkpoints', 'BS_CONFIGURATION_202309251452.csv', 'DATE_1M_2023-10-20.csv', 'KIER 전처리 현황_2024-06-25.xlsx', 'KIER_0_Raw', 'KIER_1_Cleansed', 'KIER_2_BLD', 'KIER_3_H_ELEC', 'KIER_3_H_GAS', 'KIER_3_H_HEAT', 'KIER_3_H_HOT_FLOW', 'KIER_3_H_HOT_HEAT', 'KIER_3_H_WATER', 'KIER_ASOS_WEATHER_DAILY_202309251521.csv', 'KIER_ASOS_WEATHER_HOUR_202309251521.csv', 'KIER_DATA_OLD', 'KIER_ETC', 'KIER_hList_Comparison_2024-06-26.xlsx', 'KIER_List_Table_Column_2023-09-25.xlsx', 'KIER_Query_2023-09-25.txt', 'KMA_ASOS_119_2010_2023_1st_to CSV.csv', '[IITP] 데이터 테이블 정리 (공유 원본).docx']

['KIER_GAS_561-1-1_ACCU_01_Raw.csv', 'KIER_GAS_561-1-1_INST_01_10min.csv', 'KIER_GAS_561-1-2_ACCU_01_Raw.csv', 'KIER_GAS_561-1-2_INST_01_10min.csv', 'KIER_GAS_561-1-3_ACCU_01_Raw.csv', 'KIER_GAS_561-1-3_INST_01_10min.csv', 'KIER_GAS_561-1-4_ACCU_01_Raw.csv', 'KIER_GAS_561-1-4_INST_01_10min.csv', 'KIER_GAS_561-10-1_ACCU_01_Raw.csv', 'KIER_GAS_561-10-1_INST_01_10min.csv', 'KIER_GAS_561-10-2_ACCU_01_Raw.csv', 'KIE

#### 02-01. KIER (hList)

In [71]:
## "KIER_01-01_Data_hList.ipynb"로부터 만들어진 Bld/F/H List
df_kier_hList = pd.read_csv(str_dir_raw + str_fileRaw_hList, index_col = 0)
# print(df_kier_hList.shape, ' /// ', df_kier_hList.columns)
list(df_kier_hList['HOUSE_ID'])
# df_kier_hList['HOUSE_ID']

['561-1-1',
 '561-1-2',
 '561-1-3',
 '561-1-4',
 '561-2-1',
 '561-2-2',
 '561-2-3',
 '561-2-4',
 '561-3-1',
 '561-3-2',
 '561-3-3',
 '561-3-4',
 '561-4-1',
 '561-4-2',
 '561-4-3',
 '561-4-4',
 '561-5-1',
 '561-5-2',
 '561-5-3',
 '561-5-4',
 '561-6-1',
 '561-6-2',
 '561-6-3',
 '561-6-4',
 '561-7-1',
 '561-7-2',
 '561-7-3',
 '561-7-4',
 '561-8-1',
 '561-8-2',
 '561-8-3',
 '561-8-4',
 '561-9-1',
 '561-9-2',
 '561-9-3',
 '561-9-4',
 '561-10-1',
 '561-10-2',
 '561-10-3',
 '561-10-4',
 '561-11-1',
 '561-11-2',
 '561-11-3',
 '561-11-4',
 '561-12-1',
 '561-12-2',
 '561-12-3',
 '561-12-4',
 '561-13-1',
 '561-13-2',
 '561-13-3',
 '561-13-4',
 '561-14-1',
 '561-14-2',
 '561-14-3',
 '561-14-4',
 '561-15-1',
 '561-15-2',
 '561-15-3',
 '561-15-4',
 '561-16-1',
 '561-16-2',
 '561-16-3',
 '561-16-4',
 '561-17-1',
 '561-17-2',
 '561-17-3',
 '561-17-4',
 '561-18-1',
 '561-18-2',
 '561-18-3',
 '561-18-4',
 '562-1-1',
 '562-1-2',
 '562-1-3',
 '562-1-4',
 '562-1-5',
 '562-1-6',
 '562-2-1',
 '562-2-2',
 '56

## 01-02. Data Load (df_raw)

### 01-02-01. KIER (Energy Usage : INST Interpolated)

In [72]:
## Load Intergrated ACCU Usage
str_file_accu = 'KIER_' + str_domain + '_ACCU_10MIN.csv'
## Load Intergrated INST Usage
str_file_inst = 'KIER_' + str_domain + '_INST_10MIN_2st_Linear.csv'

df_accu = pd.read_csv(str_dirName_h + str_file_accu, index_col = 0)
df_inst = pd.read_csv(str_dirName_h + str_file_inst, index_col = 0)

# df_INST_Intergrated = com_date.create_col_datetime(df_INST_Intergrated, 'METER_DATE', 'YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE').drop(labels=['None'], axis = 1)

## 각 호실별 사용량
list_col_tar_accu = list(df_accu.columns)[5:-2]
list_col_tar_inst = list(df_inst.columns)[5:-2]

print(df_accu.isna().sum().sum())
print(df_inst.isna().sum().sum())
df_accu
df_inst

2885958
1347489


,YEAR,MONTH,DAY,HOUR,MINUTE,GAS_INST_FLOW_561-1-1,GAS_INST_FLOW_561-1-2,GAS_INST_FLOW_561-1-3,GAS_INST_FLOW_561-1-4,GAS_INST_FLOW_561-2-1,...,GAS_INST_FLOW_563-22-5,GAS_INST_FLOW_563-22-6,GAS_INST_FLOW_563-23-1,GAS_INST_FLOW_563-23-2,GAS_INST_FLOW_563-23-3,GAS_INST_FLOW_563-23-4,GAS_INST_FLOW_563-24-1,GAS_INST_FLOW_563-24-2,MEAN_OF_INST,SUM_OF_INST
0,2022,8,26,20,50,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000000000
1,2022,8,26,21,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000000000
2,2022,8,26,21,10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000000000
3,2022,8,26,21,20,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000000000
4,2022,8,26,21,30,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96124,2024,6,24,9,30,0.0000000000,0.0000000000,0.0000000000,0.0004129794,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0004129794,0.1400000000
96125,2024,6,24,9,40,0.0000000000,0.0000000000,0.0000000000,0.0002409639,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0002409639,0.0800000000
96126,2024,6,24,9,50,0.0000000000,0.0000000000,0.0000000000,0.0003951368,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0003951368,0.1300000000
96127,2024,6,24,10,0,0.0000000000,0.0003539823,0.0003539823,0.0003539823,0.0003539823,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0003539823,0.0800000000


In [73]:
## "평균대치" 보간 (확인차 재보간)
## 1) ELEC - 30분 정도 소요
for i in range(0, len(df_inst)) : df_inst.loc[i] = df_inst.loc[i].fillna(df_inst['MEAN_OF_INST'].loc[i])

In [74]:
## "단순Linear -> 단순 0 대입" 보간 (확인차 재보간)
for HID in list(df_kier_hList['HOUSE_ID']) : df_inst[str_col_inst + '_' + HID] = df_inst[str_col_inst + '_' + HID].interpolate()
df_inst = df_inst.fillna(0)
df_inst

,YEAR,MONTH,DAY,HOUR,MINUTE,GAS_INST_FLOW_561-1-1,GAS_INST_FLOW_561-1-2,GAS_INST_FLOW_561-1-3,GAS_INST_FLOW_561-1-4,GAS_INST_FLOW_561-2-1,...,GAS_INST_FLOW_563-22-5,GAS_INST_FLOW_563-22-6,GAS_INST_FLOW_563-23-1,GAS_INST_FLOW_563-23-2,GAS_INST_FLOW_563-23-3,GAS_INST_FLOW_563-23-4,GAS_INST_FLOW_563-24-1,GAS_INST_FLOW_563-24-2,MEAN_OF_INST,SUM_OF_INST
0,2022,8,26,20,50,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
1,2022,8,26,21,0,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
2,2022,8,26,21,10,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
3,2022,8,26,21,20,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
4,2022,8,26,21,30,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96124,2024,6,24,9,30,0.0000000000,0.0000000000,0.0000000000,0.0004129794,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0004129794,0.1400000000
96125,2024,6,24,9,40,0.0000000000,0.0000000000,0.0000000000,0.0002409639,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0002409639,0.0800000000
96126,2024,6,24,9,50,0.0000000000,0.0000000000,0.0000000000,0.0003951368,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0003951368,0.1300000000
96127,2024,6,24,10,0,0.0000000000,0.0003539823,0.0003539823,0.0003539823,0.0003539823,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0003539823,0.0800000000


In [75]:
df_accu

,YEAR,MONTH,DAY,HOUR,MINUTE,GAS_ACCU_FLOW_561-1-1,GAS_ACCU_FLOW_561-1-2,GAS_ACCU_FLOW_561-1-3,GAS_ACCU_FLOW_561-1-4,GAS_ACCU_FLOW_561-2-1,...,GAS_ACCU_FLOW_563-22-3,GAS_ACCU_FLOW_563-22-4,GAS_ACCU_FLOW_563-22-5,GAS_ACCU_FLOW_563-22-6,GAS_ACCU_FLOW_563-23-1,GAS_ACCU_FLOW_563-23-2,GAS_ACCU_FLOW_563-23-3,GAS_ACCU_FLOW_563-23-4,GAS_ACCU_FLOW_563-24-1,GAS_ACCU_FLOW_563-24-2
0,2022,8,26,20,50,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0000000000,NaN,NaN,NaN
1,2022,8,26,21,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022,8,26,21,10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022,8,26,21,20,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022,8,26,21,30,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96124,2024,6,24,9,30,0.1300000000,0.1300000000,116.2500000000,NaN,0.1100000000,...,107.1100000000,80.5600000000,98.2200000000,31.0500000000,0.1000000000,0.1200000000,119.8000000000,123.3100000000,159.0500000000,0.1100000000
96125,2024,6,24,9,40,0.1300000000,0.1300000000,116.2500000000,NaN,0.1100000000,...,107.1100000000,80.5600000000,98.2200000000,31.0500000000,0.1000000000,0.1200000000,119.8000000000,123.3100000000,159.0500000000,0.1100000000
96126,2024,6,24,9,50,0.1300000000,0.1300000000,116.2500000000,NaN,0.1100000000,...,107.1100000000,80.5600000000,98.2200000000,31.0500000000,0.1000000000,0.1200000000,119.8000000000,123.3100000000,159.0500000000,0.1100000000
96127,2024,6,24,10,0,0.1300000000,0.1300000000,116.2500000000,NaN,0.1100000000,...,107.1100000000,80.5600000000,98.2200000000,31.0500000000,0.1000000000,0.1200000000,119.8000000000,123.3100000000,159.0500000000,0.1100000000


In [76]:
## 각 호실별 최소 적산치를 첫 행의 에너지 사용량에 대입
for HID in list(df_kier_hList['HOUSE_ID']) :
    if math.isnan(df_accu[str_col_accu + '_' + HID].loc[0]) : df_accu[str_col_accu + '_' + HID].iloc[0] = df_accu[str_col_accu + '_' + HID].min()
df_accu

,YEAR,MONTH,DAY,HOUR,MINUTE,GAS_ACCU_FLOW_561-1-1,GAS_ACCU_FLOW_561-1-2,GAS_ACCU_FLOW_561-1-3,GAS_ACCU_FLOW_561-1-4,GAS_ACCU_FLOW_561-2-1,...,GAS_ACCU_FLOW_563-22-3,GAS_ACCU_FLOW_563-22-4,GAS_ACCU_FLOW_563-22-5,GAS_ACCU_FLOW_563-22-6,GAS_ACCU_FLOW_563-23-1,GAS_ACCU_FLOW_563-23-2,GAS_ACCU_FLOW_563-23-3,GAS_ACCU_FLOW_563-23-4,GAS_ACCU_FLOW_563-24-1,GAS_ACCU_FLOW_563-24-2
0,2022,8,26,20,50,0.1200000000,0.1300000000,1.5400000000,0.1400000000,0.1100000000,...,1.5900000000,0.9100000000,1.7300000000,0.9000000000,0.1000000000,0.1200000000,0.0000000000,0.0000000000,1.5300000000,0.1100000000
1,2022,8,26,21,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022,8,26,21,10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022,8,26,21,20,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022,8,26,21,30,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96124,2024,6,24,9,30,0.1300000000,0.1300000000,116.2500000000,NaN,0.1100000000,...,107.1100000000,80.5600000000,98.2200000000,31.0500000000,0.1000000000,0.1200000000,119.8000000000,123.3100000000,159.0500000000,0.1100000000
96125,2024,6,24,9,40,0.1300000000,0.1300000000,116.2500000000,NaN,0.1100000000,...,107.1100000000,80.5600000000,98.2200000000,31.0500000000,0.1000000000,0.1200000000,119.8000000000,123.3100000000,159.0500000000,0.1100000000
96126,2024,6,24,9,50,0.1300000000,0.1300000000,116.2500000000,NaN,0.1100000000,...,107.1100000000,80.5600000000,98.2200000000,31.0500000000,0.1000000000,0.1200000000,119.8000000000,123.3100000000,159.0500000000,0.1100000000
96127,2024,6,24,10,0,0.1300000000,0.1300000000,116.2500000000,NaN,0.1100000000,...,107.1100000000,80.5600000000,98.2200000000,31.0500000000,0.1000000000,0.1200000000,119.8000000000,123.3100000000,159.0500000000,0.1100000000


In [77]:
## 각 호실별 보간된 순시사용량을 적산치에 적용 (80~90분 정도 소요)
for HID in list(df_kier_hList['HOUSE_ID']) :
    print(HID)
    ## 두번째 행부터 적용
    for i in range(1, len(df_accu)) : df_accu[str_col_accu + '_' + HID].loc[i] = df_accu[str_col_accu + '_' + HID].loc[i - 1] + df_inst[str_col_inst + '_' + HID].loc[i - 1]
df_accu

561-1-1
561-1-2
561-1-3
561-1-4
561-2-1
561-2-2
561-2-3
561-2-4
561-3-1
561-3-2
561-3-3
561-3-4
561-4-1
561-4-2
561-4-3
561-4-4
561-5-1
561-5-2
561-5-3
561-5-4
561-6-1
561-6-2
561-6-3
561-6-4
561-7-1
561-7-2
561-7-3
561-7-4
561-8-1
561-8-2
561-8-3
561-8-4
561-9-1
561-9-2
561-9-3
561-9-4
561-10-1
561-10-2
561-10-3
561-10-4
561-11-1
561-11-2
561-11-3
561-11-4
561-12-1
561-12-2
561-12-3
561-12-4
561-13-1
561-13-2
561-13-3
561-13-4
561-14-1
561-14-2
561-14-3
561-14-4
561-15-1
561-15-2
561-15-3
561-15-4
561-16-1
561-16-2
561-16-3
561-16-4
561-17-1
561-17-2
561-17-3
561-17-4
561-18-1
561-18-2
561-18-3
561-18-4
562-1-1
562-1-2
562-1-3
562-1-4
562-1-5
562-1-6
562-2-1
562-2-2
562-2-3
562-2-4
562-2-5
562-2-6
562-3-1
562-3-2
562-3-3
562-3-4
562-3-5
562-3-6
562-4-1
562-4-2
562-4-3
562-4-4
562-4-5
562-4-6
562-5-1
562-5-2
562-5-3
562-5-4
562-5-5
562-5-6
562-6-1
562-6-2
562-6-3
562-6-4
562-6-5
562-6-6
562-7-1
562-7-2
562-7-3
562-7-4
562-7-5
562-7-6
562-8-1
562-8-2
562-8-3
562-8-4
562-8-5
562-8-6
562-

,YEAR,MONTH,DAY,HOUR,MINUTE,GAS_ACCU_FLOW_561-1-1,GAS_ACCU_FLOW_561-1-2,GAS_ACCU_FLOW_561-1-3,GAS_ACCU_FLOW_561-1-4,GAS_ACCU_FLOW_561-2-1,...,GAS_ACCU_FLOW_563-22-3,GAS_ACCU_FLOW_563-22-4,GAS_ACCU_FLOW_563-22-5,GAS_ACCU_FLOW_563-22-6,GAS_ACCU_FLOW_563-23-1,GAS_ACCU_FLOW_563-23-2,GAS_ACCU_FLOW_563-23-3,GAS_ACCU_FLOW_563-23-4,GAS_ACCU_FLOW_563-24-1,GAS_ACCU_FLOW_563-24-2
0,2022,8,26,20,50,0.1200000000,0.1300000000,1.5400000000,0.1400000000,0.1100000000,...,1.5900000000,0.9100000000,1.7300000000,0.9000000000,0.1000000000,0.1200000000,0.0000000000,0.0000000000,1.5300000000,0.1100000000
1,2022,8,26,21,0,0.1200000000,0.1300000000,1.5400000000,0.1400000000,0.1100000000,...,1.5900000000,0.9100000000,1.7300000000,0.9000000000,0.1000000000,0.1200000000,0.0000000000,0.0000000000,1.5300000000,0.1100000000
2,2022,8,26,21,10,0.1200000000,0.1300000000,1.5400000000,0.1400000000,0.1100000000,...,1.5900000000,0.9100000000,1.7300000000,0.9000000000,0.1000000000,0.1200000000,0.0000000000,0.0000000000,1.5300000000,0.1100000000
3,2022,8,26,21,20,0.1200000000,0.1300000000,1.5400000000,0.1400000000,0.1100000000,...,1.5900000000,0.9100000000,1.7300000000,0.9000000000,0.1000000000,0.1200000000,0.0000000000,0.0000000000,1.5300000000,0.1100000000
4,2022,8,26,21,30,0.1200000000,0.1300000000,1.5400000000,0.1400000000,0.1100000000,...,1.5900000000,0.9100000000,1.7300000000,0.9000000000,0.1000000000,0.1200000000,0.0000000000,0.0000000000,1.5300000000,0.1100000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96124,2024,6,24,9,30,1.2265582811,4.9229868879,137.6671362594,7.1795856025,41.0541948492,...,101.6335426693,81.4748031782,96.1834781938,32.0079067537,4.8163148360,3.4613273905,118.1774391570,127.1390819162,1990.3292276733,3.7353104377
96125,2024,6,24,9,40,1.2265582811,4.9229868879,137.6671362594,7.1799985819,41.0541948492,...,101.6335426693,81.4748031782,96.1834781938,32.0079067537,4.8163148360,3.4613273905,118.1774391570,127.1390819162,1990.3292276733,3.7353104377
96126,2024,6,24,9,50,1.2265582811,4.9229868879,137.6671362594,7.1802395457,41.0541948492,...,101.6335426693,81.4748031782,96.1834781938,32.0079067537,4.8163148360,3.4613273905,118.1774391570,127.1390819162,1990.3292276733,3.7353104377
96127,2024,6,24,10,0,1.2265582811,4.9229868879,137.6671362594,7.1806346825,41.0541948492,...,101.6335426693,81.4748031782,96.1834781938,32.0079067537,4.8163148360,3.4613273905,118.1774391570,127.1390819162,1990.3292276733,3.7353104377


In [78]:
df_accu = com_date.create_col_datetime(df_accu, 'METER_DATE', 'YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE')
df_accu

,METER_DATE,YEAR,MONTH,DAY,HOUR,MINUTE,GAS_ACCU_FLOW_561-1-1,GAS_ACCU_FLOW_561-1-2,GAS_ACCU_FLOW_561-1-3,GAS_ACCU_FLOW_561-1-4,...,GAS_ACCU_FLOW_563-22-4,GAS_ACCU_FLOW_563-22-5,GAS_ACCU_FLOW_563-22-6,GAS_ACCU_FLOW_563-23-1,GAS_ACCU_FLOW_563-23-2,GAS_ACCU_FLOW_563-23-3,GAS_ACCU_FLOW_563-23-4,GAS_ACCU_FLOW_563-24-1,GAS_ACCU_FLOW_563-24-2,None
0,2022-08-26 20:50:00,2022,8,26,20,50,0.1200000000,0.1300000000,1.5400000000,0.1400000000,...,0.9100000000,1.7300000000,0.9000000000,0.1000000000,0.1200000000,0.0000000000,0.0000000000,1.5300000000,0.1100000000,0
1,2022-08-26 21:00:00,2022,8,26,21,0,0.1200000000,0.1300000000,1.5400000000,0.1400000000,...,0.9100000000,1.7300000000,0.9000000000,0.1000000000,0.1200000000,0.0000000000,0.0000000000,1.5300000000,0.1100000000,0
2,2022-08-26 21:10:00,2022,8,26,21,10,0.1200000000,0.1300000000,1.5400000000,0.1400000000,...,0.9100000000,1.7300000000,0.9000000000,0.1000000000,0.1200000000,0.0000000000,0.0000000000,1.5300000000,0.1100000000,0
3,2022-08-26 21:20:00,2022,8,26,21,20,0.1200000000,0.1300000000,1.5400000000,0.1400000000,...,0.9100000000,1.7300000000,0.9000000000,0.1000000000,0.1200000000,0.0000000000,0.0000000000,1.5300000000,0.1100000000,0
4,2022-08-26 21:30:00,2022,8,26,21,30,0.1200000000,0.1300000000,1.5400000000,0.1400000000,...,0.9100000000,1.7300000000,0.9000000000,0.1000000000,0.1200000000,0.0000000000,0.0000000000,1.5300000000,0.1100000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96124,2024-06-24 09:30:00,2024,6,24,9,30,1.2265582811,4.9229868879,137.6671362594,7.1795856025,...,81.4748031782,96.1834781938,32.0079067537,4.8163148360,3.4613273905,118.1774391570,127.1390819162,1990.3292276733,3.7353104377,0
96125,2024-06-24 09:40:00,2024,6,24,9,40,1.2265582811,4.9229868879,137.6671362594,7.1799985819,...,81.4748031782,96.1834781938,32.0079067537,4.8163148360,3.4613273905,118.1774391570,127.1390819162,1990.3292276733,3.7353104377,0
96126,2024-06-24 09:50:00,2024,6,24,9,50,1.2265582811,4.9229868879,137.6671362594,7.1802395457,...,81.4748031782,96.1834781938,32.0079067537,4.8163148360,3.4613273905,118.1774391570,127.1390819162,1990.3292276733,3.7353104377,0
96127,2024-06-24 10:00:00,2024,6,24,10,0,1.2265582811,4.9229868879,137.6671362594,7.1806346825,...,81.4748031782,96.1834781938,32.0079067537,4.8163148360,3.4613273905,118.1774391570,127.1390819162,1990.3292276733,3.7353104377,0


In [79]:
df_accu.isna().sum().sum()

0

In [80]:
str_file_accu = 'KIER_' + str_domain + '_ACCU_10MIN_Restored.csv'
df_accu.to_csv(str_dirName_h + str_file_accu)